In [ ]:
import pandas as pd
longNQ = pd.read_json("/dccstor/srosent2/generative/appen/NQ_formatted_answered_multiple-9.11.23.json", orient='records', lines=True, dtype={"id":str})
longNQ.sample(5)
longNQTrain = pd.read_json("/dccstor/srosent2/generative/appen/NQ_formatted_answered_multiple-9.11.23_train.json", orient='records', lines=True, dtype={"id":str})
longNQDev = pd.read_json("/dccstor/srosent2/generative/appen/NQ_formatted_answered_multiple-9.11.23_dev.json", orient='records', lines=True, dtype={"id":str})


In [ ]:


longNQ_first_word  = longNQ['input'].str.split().str.get(0)
long_NQ_first_word_counts = longNQ_first_word.value_counts()[0:20] #/len(longNQ)
print(long_NQ_first_word_counts.sum())
long_NQ_first_word_counts['OTHER'] = 2379-long_NQ_first_word_counts.sum()
print(long_NQ_first_word_counts.sum())
print(long_NQ_first_word_counts['OTHER'])
word_dist = []

for word in long_NQ_first_word_counts.keys():
    occurrence = long_NQ_first_word_counts[word]
    word_dist.extend([word]*occurrence)

print(long_NQ_first_word_counts)
print(len(word_dist))

In [ ]:
from random import *

print(word_dist)
random_num = randint(0,len(word_dist))
print(random_num)
print(word_dist[random_num])


In [ ]:
import glob

originalNQfiles = glob.glob("/dccstor/srosent2/primeqa/data/train/nq-full/*")

originalNQ = pd.concat([pd.read_json(f,orient='records',lines=True, dtype={"example_id":str}) for f in originalNQfiles])

originalNQ.head(5)


In [ ]:
import glob

originalNQdevfiles = glob.glob("/dccstor/srosent2/primeqa/data/dev/nq-full/*")

originalNQdev = pd.concat([pd.read_json(f,orient='records',lines=True, dtype={"example_id":str}) for f in originalNQdevfiles])

originalNQdev.head(5)


In [ ]:
# filter to only negatives
negativeNQids1word = {}
for i, row in originalNQ.iterrows():
    if row['annotations'][0]['passage_answer']['candidate_index'] == -1:
        first_word = row['question_text'].split()[0]

        if first_word not in word_dist:
            first_word = "OTHER"
        if first_word not in negativeNQids1word:
            negativeNQids1word[first_word] = set()
        negativeNQids1word[first_word].add(row['example_id'])
        
print(len(negativeNQids1word))

In [ ]:
# pre-existing negs
import pandas as pd

existing_data = pd.read_csv('/dccstor/srosent2/generative/hans-dev-questions.tsv',delimiter='\t')

existing_negs = existing_data[existing_data['relevant'] == '-1']
existing_pos = existing_data[existing_data['relevant'] != '-1']
print(existing_negs.sample(5))
print(len(existing_negs))

In [ ]:
# filter to only negatives
import pprint 

def filter_original(thisNQ, existing, fname, id_key, thisLongNQ):
    existingids1worddev = {}
    dev_ids = set()

    for i, row in existing.iterrows():
        first_word = row['text'].split()[0]
        existing = False

        # check if this question is in existing negs.
        recordNQ = thisNQ[thisNQ['question_text'] == row['text']]
        if len(recordNQ) > 0:
            existing = True
            dev_ids.add(recordNQ.iloc[0]['example_id'])

            if first_word not in word_dist:
                first_word = "OTHER"
            if first_word not in existingids1worddev:
                existingids1worddev[first_word] = 0
            existingids1worddev[first_word] += 1
    print(len(existingids1worddev))

    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(existingids1worddev)
    print(len(dev_ids))
    dataNQ= thisLongNQ[thisLongNQ[id_key].isin(list(dev_ids))]
    dataNQ.to_json(f"/dccstor/srosent2/generative/appen/final/{fname}.jsonl", orient='records', lines=True)
    return dev_ids

used_ids = []
used_ids.extend(filter_original(originalNQdev, existing_negs, 'longNQ_dev_unanswerable_tydi', 'example_id', originalNQdev))
used_ids.extend(filter_original(originalNQ, existing_pos, 'longNQ_dev_answerable_tydi', 'example_id', originalNQ))
used_ids.extend(filter_original(originalNQ, existing_pos, 'longNQ_dev_answerable_eli5', 'id', longNQ))



In [ ]:
# Train: 1813
# Dev: 300: 300 train (from hans)
# Test: 67 dev + 233 more
# NA train: 1813 from NQ train following distribution
# NA dev: 300 from hans 
# NA test: 300 following distribution

In [ ]:
# NA train: 1813

from random import *

na_ids = set()
na_train_words = {}

while len(na_ids) < 20379:
    random_num = randint(0,len(word_dist)-1)
    random_id = list(negativeNQids1word[word_dist[random_num]])[randint(0,len(negativeNQids1word[word_dist[random_num]])-1)]
    num_tries = 5
    while random_id in na_ids and num_tries > 0:
        random_id = list(negativeNQids1word[word_dist[random_num]])[randint(0,len(negativeNQids1word[word_dist[random_num]])-1)]
        num_tries -= 1
    if random_id in na_ids:
        print("retry")
        continue
    if word_dist[random_num] not in na_train_words:
        na_train_words[word_dist[random_num]] = 0
    na_train_words[word_dist[random_num]] += 1
    na_ids.add(random_id)
print(len(na_ids))
print(na_train_words)


In [ ]:
import pprint 
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(na_train_words)

In [ ]:
# NA train: 1791 from NQ train following distribution but check to make sure there are 5 sentences.
import spacy 

# spacy.cli.download('en_core_web_sm')
nlp = spacy.load('en_core_web_sm')

trainNQNA = originalNQ[originalNQ['example_id'].isin(na_ids)] #[:1791]
long_par_ids = []

count = 0
for i, row in trainNQNA.iterrows():
    annotation = row['annotations'][0]
    passage_offsets = row['passage_answer_candidates'][annotation['passage_answer']['candidate_index']]
    passage_text = row['document_plaintext'].encode('utf-8')[passage_offsets['plaintext_start_byte']:passage_offsets['plaintext_end_byte']].decode('utf-8')
    sentences = nlp(passage_text)
        
    num_sentences = 0
    for sentence in sentences.sents:
        num_sentences += 1
    if num_sentences < 5:
        continue
    long_par_ids.append(row['example_id'])
    count += 1
    if count >= 2000:
        break
    

trainNQNA = trainNQNA[trainNQNA['example_id'].isin(long_par_ids)][:1791]
trainNQNA.to_json("/dccstor/srosent2/generative/appen/final/longNQ_train_unanswerable_tydi.jsonl", orient='records', lines=True)


In [ ]:
# make test sets 
negativeNQids1worddev = {}
existing_negsids1worddev = {}

def allNA(annotations):
    count = 0
    for annotation in annotations:
        if annotation['passage_answer']['candidate_index'] != -1:
            return False
    return True

num_rows = 0
for i, row in originalNQdev.iterrows():
    if allNA(row['annotations']):
        first_word = row['question_text'].split()[0]
        existing = False

        # check if this question is in existing negs.
        if len(existing_negs[row['question_text'] == existing_negs['text']]) > 0:
            existing = True

        if first_word not in word_dist:
            first_word = "OTHER"
        if first_word not in negativeNQids1worddev:
            negativeNQids1worddev[first_word] = set()
        if existing and first_word not in existing_negsids1worddev:
            existing_negsids1worddev[first_word] = set()
        
        if existing: 
            existing_negsids1worddev[first_word].add(row['example_id'])
        else:
            negativeNQids1worddev[first_word].add(row['example_id'])
            num_rows += 1
        
print(num_rows)
print(len(negativeNQids1worddev))
print(len(existing_negsids1worddev))

from random import *

na_dev_ids = set()
na_dev_words = {}

while len(na_dev_ids) < 2139:
    random_num = randint(0,len(word_dist)-1)
    random_id = list(negativeNQids1worddev[word_dist[random_num]])[randint(0,len(negativeNQids1worddev[word_dist[random_num]])-1)]

    if random_id in used_ids:
        continue
    num_tries = 5
    while random_id in na_ids and num_tries > 0:
        random_id = list(negativeNQids1worddev[word_dist[random_num]])[randint(0,len(negativeNQids1worddev[word_dist[random_num]])-1)]
        num_tries -= 1
    if random_id in na_dev_ids:
        print("retry")
        continue
    if word_dist[random_num] not in na_dev_words:
        na_dev_words[word_dist[random_num]] = 0
    na_dev_words[word_dist[random_num]] += 1
    na_dev_ids.add(random_id)
print(len(na_dev_ids))
print(na_dev_words)

testNQNA = originalNQdev[originalNQdev['example_id'].isin(na_dev_ids)]

import spacy 

# spacy.cli.download('en_core_web_sm')
nlp = spacy.load('en_core_web_sm')

long_par_ids = []

count = 0
for i, row in testNQNA.iterrows():
    annotation = row['annotations'][0]
    passage_offsets = row['passage_answer_candidates'][annotation['passage_answer']['candidate_index']]
    passage_text = row['document_plaintext'].encode('utf-8')[passage_offsets['plaintext_start_byte']:passage_offsets['plaintext_end_byte']].decode('utf-8')
    sentences = nlp(passage_text)
        
    num_sentences = 0
    for sentence in sentences.sents:
        num_sentences += 1
    if num_sentences < 5:
        continue
    long_par_ids.append(row['example_id'])
    count += 1
    if count >= 2139:
        break

testNQNA = testNQNA[testNQNA['example_id'].isin(long_par_ids)][:300]
testNQNA.to_json("/dccstor/srosent2/generative/appen/final/longNQ_test_unanswerable_tydi.jsonl", orient='records', lines=True)

In [ ]:
# Train: 1813
# Test: 67 dev + 233 more

longNQTrain = pd.read_json("/dccstor/srosent2/generative/appen/NQ_formatted_answered_multiple-9.15.23_train.json", orient='records', lines=True, dtype={"id":str})
longNQTrain = longNQTrain[~longNQTrain['id'].isin(used_ids)]
longNQDev = pd.read_json("/dccstor/srosent2/generative/appen/NQ_formatted_answered_multiple-9.15.23_dev.json", orient='records', lines=True, dtype={"id":str})
longNQDev = longNQDev[~longNQDev['id'].isin(used_ids)]

print(len(longNQTrain))
print(len(longNQDev))

longNQTrainShuffled = longNQTrain.sample(frac=1)

longNQTrainNew = longNQTrainShuffled[234:]
longNQDevNew = pd.concat([longNQTrainShuffled[0:234],longNQDev])

longNQTrainNew.to_json(f"/dccstor/srosent2/generative/appen/final/longNQ_train_answerable_eli5.jsonl", orient='records', lines=True)
longNQDevNew.to_json(f"/dccstor/srosent2/generative/appen/final/longNQ_test_answerable_eli5.jsonl", orient='records', lines=True)
originalNQ[originalNQ['example_id'].isin(list(longNQTrainNew['id']))].to_json(f"/dccstor/srosent2/generative/appen/final/longNQ_train_answerable_tydi.jsonl", orient='records', lines=True)
pd.concat([originalNQ[originalNQ['example_id'].isin(list(longNQDevNew['id']))],originalNQdev[originalNQdev['example_id'].isin(list(longNQDevNew['id']))]]).to_json(f"/dccstor/srosent2/generative/appen/final/longNQ_test_answerable_tydi.jsonl", orient='records', lines=True)


In [ ]:
# check that splits are compatible with original NQ
import pandas as pd

final_files = glob.glob("/dccstor/srosent2/generative/appen/final/*/*/*")

unique_ids = set()

for final_file in final_files:
    id_key = 'id'
    if 'tydi' in final_file:
        id_key = 'example_id'
    data = pd.read_json(final_file, orient='records', lines=True, dtype={id_key:str})

    print(final_file)
    print(f"file: {len(data)}")
    print(f"train: {len(originalNQ[originalNQ['example_id'].isin(list(data[id_key]))])}")
    print(f"dev: {len(originalNQdev[originalNQdev['example_id'].isin(list(data[id_key]))])}")
    unique_ids.update(list(data[id_key]))
print(len(unique_ids))

In [ ]:
# check that splits are compatible with original NQ
import pandas as pd

longNQTrain = pd.read_json("/dccstor/srosent2/generative/appen/NQ_formatted_answered_multiple-9.15.23_train.json", orient='records', lines=True, dtype={"id":str})
longNQDev = pd.read_json("/dccstor/srosent2/generative/appen/NQ_formatted_answered_multiple-9.15.23_dev.json", orient='records', lines=True, dtype={"id":str})

final_files = glob.glob("/dccstor/srosent2/generative/appen/final/*/*/*_answerable*")

for final_file in final_files:
    id_key = 'id'
    if 'tydi' in final_file:
        id_key = 'example_id'
    data = pd.read_json(final_file, orient='records', lines=True, dtype={id_key:str})

    print(final_file)
    print(f"file: {len(data)}")
    print(f"train: {len(longNQTrain[longNQTrain['id'].isin(list(data[id_key]))])}")
    print(f"dev: {len(longNQDev[longNQDev['id'].isin(list(data[id_key]))])}")

In [ ]:
# check non-consecutive
# check that IDs are all unique YES
# check that TyDi and ELI5 splits have same IDs YES

import pandas as pd
import glob

final_files = glob.glob("/dccstor/srosent2/generative/appen/final/*/*/*")

seen_ids_tydi = set()
seen_ids_eli5 = set()

for final_file in final_files:
    id_key = 'id'
    if 'tydi' in final_file:
        id_key = 'example_id'
        data = pd.read_json(final_file, orient='records', lines=True, dtype={id_key:str})
        seen_ids_tydi.update(data[id_key])
    else:
        data = pd.read_json(final_file, orient='records', lines=True, dtype={id_key:str})
        seen_ids_eli5.update(data[id_key])

        if "_answerable" in final_file:
            num_false = 0
            for i, row in data.iterrows():
                if row['output'][0]['meta']['non_consecutive'] == False:
                    num_false += 1
            print(f"{final_file}: {num_false}")

print(len(seen_ids_tydi))
print(len(seen_ids_eli5))

print(seen_ids_tydi == seen_ids_eli5)



In [ ]:
# add to test set whether from original NQ dev or test

# check that splits are compatible with original NQ
import pandas as pd

id_key = 'id'
data = pd.read_json("/dccstor/srosent2/generative/appen/final/longNQ/test/longNQ_test_answerable.jsonl", orient='records', lines=True, dtype={id_key:str})

for i, row in data.iterrows():
    if len(longNQDev[longNQDev['id'] == row[id_key]]) > 0:
        data.at[i, 'is_dev'] = True
    else:
        data.at[i, 'is_dev'] = False

data.to_json("/dccstor/srosent2/generative/appen/final/longNQ/test/longNQ_test_answerable_wdev.jsonl", orient='records', lines=True)

In [3]:
# update data to include missing fields. for loading in HF
import pandas as pd

id_key = 'id'

def convert_to_hf(data):
    for i, row in data.iterrows():
        for output in row['output']:
            if output['answer'] == None:
                output['answer'] = ""
            if 'selected_sentences' not in output or output['selected_sentences'] == None:
                output['selected_sentences'] = []
            if 'meta' not in output or output['meta'] == None:
                output['meta'] = {}
            if 'skip' not in output['meta']:
                output['meta']['skip'] = False
            if 'non_consecutive' not in output['meta']:
                output['meta']['non_consecutive'] = False # this may not be accurate
            if 'round' not in output['meta']:
                output['meta']['round'] = 0 # unanswerable
            if 'annotator' not in output['meta']:
                output['meta']['annotator'] = [] # unanswerable
            if 'has_minimal_answer' not in output['meta']:
                output['meta']['has_minimal_answer'] = False
            if len(output['meta']) != 5:
                print(output['meta'])
        data.at[i,'output'] = row['output']
    return data

splits = ["train", "dev", "test"]

for split in splits:
    data = pd.read_json(f"/dccstor/srosent2/generative/appen/final/longNQ/{split}/longNQ_{split}_answerable.jsonl", orient='records', lines=True, dtype={id_key:str})
    convert_to_hf(data).to_json(f"/dccstor/srosent2/generative/appen/final/longNQ_hf/{split}/longNQ_{split}_answerable.jsonl", orient='records', lines=True)
    data = pd.read_json(f"/dccstor/srosent2/generative/appen/final/longNQ/{split}/longNQ_{split}_unanswerable.jsonl", orient='records', lines=True, dtype={id_key:str})
    convert_to_hf(data).to_json(f"/dccstor/srosent2/generative/appen/final/longNQ_hf/{split}/longNQ_{split}_unanswerable.jsonl", orient='records', lines=True)

